**disallowedDeprecatedContent.ipynb**

Deprecated concept with disallowed annotations or axioms.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/disallowedDeprecatedContent.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [20]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and all properties that are *not* allowed on deprecated concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concecpt is considered deprecated if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``obsolete``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#obsolete"/>``.

**NB.3:**
Subsets in EDAM are (currently) defined in one of two ways:

``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#data" />``

``<oboInOwl:inSubset>concept_properties />``

this complicates the logic slightly.

In [21]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?is_format_of ?documentation ?is_identifier_of ?exact_syn ?narrow_syn ?broad_syn ?notRecommendedForAnnotation ?file_extension ?media_type ?example ?information_standard ?ontology_used ?organisation ?regex WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :is_format_of ?is_format_of .}
OPTIONAL {?id :documentation ?documentation .}
OPTIONAL {?id :is_identifier_of ?is_identifier_of .}
OPTIONAL {?id oboInOwl:hasExactSynonym ?exact_syn .}
OPTIONAL {?id oboInOwl:hasNarrowSynonym ?narrow_syn .}
OPTIONAL {?id oboInOwl:hasBroadSynonym ?broad_syn .}
OPTIONAL {?id rdfs:comment ?comment .}
OPTIONAL {?id :notRecommendedForAnnotation ?notRecommendedForAnnotation .}
OPTIONAL {?id :file_extension ?file_extension .}
OPTIONAL {?id :media_type ?media_type .}
OPTIONAL {?id :example ?example .}
OPTIONAL {?id :information_standard ?information_standard .}
OPTIONAL {?id :ontology_used ?ontology_used .}
OPTIONAL {?id :organisation ?organisation .}
OPTIONAL {?id :regex ?regex .}
}
"""

# Run SPARQL query and collate results
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [22]:
for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])

    is_format_of = str(r['is_format_of'])
    documentation = str(r['documentation'])
    is_identifier_of = str(r['is_identifier_of'])
    exact_syn = str(r['exact_syn'])
    narrow_syn = str(r['narrow_syn'])
    broad_syn = str(r['broad_syn'])
    notRecommendedForAnnotation = str(r['notRecommendedForAnnotation'])
    file_extension = str(r['file_extension'])
    media_type = str(r['media_type'])
    example = str(r['example'])
    information_standard = str(r['information_standard'])
    ontology_used = str(r['ontology_used'])
    organisation = str(r['organisation'])
    regex = str(r['regex'])


    # Extract subset without path, e.g. "data"
    pos = subsetpath.rfind("#")
    if pos == -1:
        # subset path does not contain '#'
        subset = subsetpath
    else:
        subset = subsetpath[pos+1:]

    # Check for (and skip) obsolete concepts
    if subset == "obsolete":
        if is_format_of != "None": 
            report.append("Invalid property (is_format_of) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue    
        if documentation != "None":
            report.append("Invalid property (documentation) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if is_identifier_of != "None":
            report.append("Invalid property (is_identifier_of) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if exact_syn  != "None":
            report.append("Invalid property (exact_syn) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if narrow_syn  != "None":
            report.append("Invalid property (narrow_syn) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if broad_syn  != "None":
            report.append("Invalid property (broad_syn) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if notRecommendedForAnnotation != "None":
            report.append("Invalid property (notRecommendedForAnnotation) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if file_extension  != "None":
            report.append("Invalid property (file_extension) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if media_type  != "None":
            report.append("Invalid property (media_type) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if example  != "None":
            report.append("Invalid property (example) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if information_standard != "None":
            report.append("Invalid property (information_standard) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if ontology_used  != "None":
            report.append("Invalid property (ontology_used) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if organisation  != "None":
            report.append("Invalid property (organisation) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue
        if regex != "None":
            report.append("Invalid property (regex) on ::: " + id +  ' (' + term + ')')
            errfound = True
            continue

Write report and return approriate value.

In [23]:
# Return exit code (raises exception)
if errfound:
    print('Invalid property found for these obsolete concepts:')
    print("\n".join(report))
    sys.exit(ERROR)
else:
    print("No issues found.")
    sys.exit(NOERR)

Invalid property found for these obsolete concepts:
Invalid property (example) on ::: http://edamontology.org/data_0985 (Molecule type)
Invalid property (regex) on ::: http://edamontology.org/data_2682 (Ensembl ID ('Gallus gallus'))
Invalid property (regex) on ::: http://edamontology.org/data_1791 (Gene name (HGNC))
Invalid property (regex) on ::: http://edamontology.org/data_2696 (Ensembl ID ('Spermophilus tridecemlineatus'))
Invalid property (regex) on ::: http://edamontology.org/data_2678 (Ensembl ID ('Dasypus novemcinctus'))
Invalid property (regex) on ::: http://edamontology.org/data_2690 (Ensembl ID ("Ornithorhynchus anatinus"))
Invalid property (regex) on ::: http://edamontology.org/data_2673 (Ensembl ID ('Canis familiaris'))
Invalid property (exact_syn) on ::: http://edamontology.org/data_1746 (Atomic z coordinate)
Invalid property (narrow_syn) on ::: http://edamontology.org/topic_3514 (Protein-ligand interactions)
Invalid property (exact_syn) on ::: http://edamontology.org/dat

SystemExit: 3

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
